In [ ]:

%load_ext autoreload
%autoreload 2

import copy
import pytorch_lightning as pl
import torchinfo
import torchvision.transforms as transforms

from PlantINaturalist2021DataModule import PlantINaturalist2021DataModule
from PlantINaturalist2021FinetuneMobileNetv2 import PlantINaturalist2021FinetuneMobileNetv2, TRANSFORM2

config = {
    "model_name": PlantINaturalist2021FinetuneMobileNetv2.__name__,
    "num_classes": 250,
    "learning_rate": 0.01,
    "lr_decay_epoch_step_size": 5,
    "lr_decay_rate": 0.9,
    "num_trainable_layers": 2,
}

# sanity run
pl.Trainer(max_steps=5).fit(model=PlantINaturalist2021FinetuneMobileNetv2(config), datamodule=PlantINaturalist2021DataModule(transform=TRANSFORM2, context = "retrain"))

model = PlantINaturalist2021FinetuneMobileNetv2(config)

datamodule = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="train", batch_size=32, num_workers=2, pin_memory=True, data_dir="./")

torchinfo.summary(model)

In [ ]:
import wandb
import torch
from pytorch_lightning.loggers import WandbLogger


run = wandb.init(project='PlantINaturalist', config=config)


In [ ]:

wandb_logger = WandbLogger()
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs=35, accelerator='gpu', devices=1)

trainer.fit(model=model, datamodule=datamodule)

In [ ]:
torch.save(model.state_dict(), 'model.pth')
artifact = wandb.Artifact('model', type='model')
artifact.add_file('model.pth')
run.log_artifact(artifact)

run.finish()

In [ ]:
next(model.model.classifier.parameters())

In [ ]:
import wandb
artifact = run.use_artifact('pasoi0stefan/PlantINaturalist/model:v11', type='model')
artifact_dir = artifact.download()

In [ ]:
model_artifact = PlantINaturalist2021FinetuneMobileNetv2(config)
model_artifact.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))

In [ ]:
#FINETUNE
wandb_logger = WandbLogger()
model_finetune = PlantINaturalist2021FinetuneMobileNetv2(config)
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.learning_rate = 0.001
model_finetune.configure_optimizers()
datamodule_finetune = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="finetune", batch_size=32, num_workers=1, pin_memory=True, data_dir="./")
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs = 20, accelerator='gpu', devices=1)
trainer.fit(model=model_finetune, datamodule=datamodule_finetune)
torch.save(model_finetune.state_dict(), 'finetuned_model.pth')
artifact = wandb.Artifact('finetuned_model', type='model')
artifact.add_file('finetuned_model.pth')
run.log_artifact(artifact)
run.finish()

In [ ]:
#FINETUNE 2
wandb_logger = WandbLogger()
model_finetune = PlantINaturalist2021FinetuneMobileNetv2(config)
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.configure_optimizers()
datamodule_finetune = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="retrain", batch_size=32, num_workers=1, pin_memory=True, data_dir="./")
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs = 10, accelerator='gpu', devices=1)
trainer.fit(model=model_finetune, datamodule=datamodule_finetune)
torch.save(model_finetune.state_dict(), 'finetuned_model2.pth')
artifact = wandb.Artifact('finetuned_model2', type='model')
artifact.add_file('finetuned_model2.pth')
run.log_artifact(artifact)
run.finish()

In [ ]:
#RETRAIN
wandb.finish()
run = wandb.init()
wandb_logger = WandbLogger()
model_retrain = PlantINaturalist2021FinetuneMobileNetv2(config)
datamodule_retrain = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="retrain", batch_size=32, num_workers=1, pin_memory=True, data_dir="./")
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs = 40, accelerator='gpu', devices=1)
trainer.fit(model=model_retrain, datamodule=datamodule_retrain)
torch.save(model_retrain.state_dict(), 'retrained_model.pth')
artifact = wandb.Artifact('retrained_model', type='model')
artifact.add_file('retrained_model.pth')
run.log_artifact(artifact)
run.finish()

In [ ]:
next(model_artifact.model.classifier.parameters())